#Продвинутый Python, ДЗ-7

Правила игры:

В домашке 1 задача. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

Оценка за ДЗ складывается следующим образом:

* наличие полного результата - 10 баллов

* наличие только 1 запроса - -4 балла

* отсутствие фильтрации - -2 балла

* запросы только с первой страницы - -2 балла

* не все поля вытащены - -2 балла

В качестве результата добавляйте в репозиторий полностью папку с пауком и всеми необходимыми файлами

**N.B.** Steam спокойно парсится без посторонних инструментов обхода блокировок

## Задание

Парсим [Steam](https://store.steampowered.com/) с помощью Scrapy!

Необходимо выбрать 3 запроса (например, "инди", "стратегии", "minecraft", на каждый запрос должно быть минимум 90 результатов), и вытащить по ним:

* Все игры на первых 2 страницах (найдите, каким образом отображаются страницы)

И для каждой игры вытащить:

* Название

* Категорию (весь путь, за исключением Все игры и самого названия игры)

* Число всех обзоров и общая оценка

* Дата выхода

* Разработчик

* Метки (тэги) игры

* Цена

* Доступные платформы

Сохраните игры в формате json, оставив только игры, выпущенные после 2000 года

Чтобы было проще, в репозитории с ДЗ выложены скрины, где можно посмотреть, где и какие поля находятся



In [ ]:
!pip install scrapy

Создаём паука:

In [ ]:
!scrapy startproject spider_steam; cd spider_steam; ls -la

In [3]:
!scrapy genspider SteamSpider store.steampowered.com

Created spider 'SteamSpider' using template 'basic' 


Пример парсинга страницы результата поиска по запросу 'animals'

In [78]:
import requests
from lxml import html

url = 'https://store.steampowered.com/search/?term=animals&ndl=1'
response = requests.get(url)
html_data = response.text

tree = html.fromstring(html_data)

games = tree.xpath('//a[@class="search_result_row ds_collapse_flag "]')

for game in games:
    link = game.xpath('./@href')[0]
    appid = game.xpath('./@data-ds-appid')
    title = game.xpath('.//span[@class="title"]/text()')
    release_date = game.xpath('.//div[@class="col search_released responsive_secondrow"]/text()')
    discount_pct = game.xpath('.//div[@class="discount_pct"]/text()')
    original_price = game.xpath('.//div[@class="discount_original_price"]/text()')
    final_price = game.xpath('.//div[@class="discount_final_price"]/text()')

    print(f'Ссылка: {link}', )
    print(f'App ID: {appid[0] if appid else 0}', )
    print(f'Название игры: {title[0] if title else 0}')
    print(f'Дата релиза: {release_date[0].strip() if release_date else 0}')
    print(f'Процент скидки: {discount_pct[0] if discount_pct else 0}')
    print(f'Исходная цена: {original_price[0] if original_price else 0}')
    print(f'Финальная цена: {final_price[0] if final_price else 0}')
    print('---')

Ссылка: https://store.steampowered.com/app/2013610/Animals/?snr=1_7_7_151_150_1
App ID: 2013610
Название игры: Animals
Дата релиза: 13 Jun, 2022
Процент скидки: 0
Исходная цена: 0
Финальная цена: 200 руб
---
Ссылка: https://store.steampowered.com/app/1260320/Party_Animals/?snr=1_7_7_151_150_1
App ID: 1260320
Название игры: Party Animals
Дата релиза: 20 Sep, 2023
Процент скидки: -30%
Исходная цена: 1238 руб
Финальная цена: 866,60 руб
---
Ссылка: https://store.steampowered.com/app/2342040/Party_Animals__Deluxe_Pack/?snr=1_7_7_151_150_1
App ID: 2342040
Название игры: Party Animals - Deluxe Pack
Дата релиза: 20 Sep, 2023
Процент скидки: -30%
Исходная цена: 619 руб
Финальная цена: 433,30 руб
---
Ссылка: https://store.steampowered.com/app/431960/Wallpaper_Engine/?snr=1_7_7_151_150_1
App ID: 431960
Название игры: Wallpaper Engine
Дата релиза: Nov 2018
Процент скидки: 0
Исходная цена: 0
Финальная цена: 249 руб
---
Ссылка: https://store.steampowered.com/app/1129580/Medieval_Dynasty/?snr=1_7_7_1

Пример парсинга ссылок на страницы продуктов(игр):

In [79]:
import requests
from bs4 import BeautifulSoup

url = "https://store.steampowered.com/search/?term=animals&ndl=1"
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

search_results = soup.find_all(class_="search_result_row")

animals = []

for result in search_results:
    product_link = result.get("href")

    # Проверяем, что ссылка ведет на сайт продукта и содержит "/app/"
    if "/app/" in product_link:
        animals.append(product_link)


In [80]:
print(animals)

['https://store.steampowered.com/app/2013610/Animals/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/1260320/Party_Animals/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/2342040/Party_Animals__Deluxe_Pack/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/431960/Wallpaper_Engine/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/1129580/Medieval_Dynasty/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/648800/Raft/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/294100/RimWorld/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/108600/Project_Zomboid/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/513710/SCUM/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/2139460/Once_Human/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/1790600/DRAGON_BALL_Sparking_ZERO/?snr=1_7_7_151_150_1', 'https://store.steampowered.com/app/1023570/PUZZLE_ANIMALS/?snr=1_7_7_151_150_1', 'https://store.steampowered.co

Как мы будем парсить отдельно страницы:

In [81]:
import requests
import lxml.html

url = 'https://store.steampowered.com/app/1260320/Party_Animals/?snr=1_7_7_151_150_1'
response = requests.get(url)
html = response.content

doc = lxml.html.fromstring(html)

name = doc.xpath('//div[@class="apphub_AppName"]/text()')[0].strip()
category = doc.xpath('//div[@class="blockbg"]/a/text()')
reviews_count = doc.xpath('//span[@class="responsive_hidden"]/text()')[1].strip()
overall_score = doc.xpath('//span[@class="game_review_summary positive"]/text()')[1]
release_date = doc.xpath('//div[@class="date"]/text()')[0].strip()
developer = doc.xpath('//div[@class="dev_row"]/a/text()')[0].strip()
tags = doc.xpath('//a[@class="app_tag"]/text()')
price = doc.xpath('//div[@class="game_purchase_price price"]/text()')[0].strip()

print('Название:', name)
print('Категория:', ' > '.join(category))
print('Число обзоров:', reviews_count)
print('Общая оценка:', overall_score)
print('Дата выхода:', release_date)
print('Разработчик:', developer)
print('Метки:', [tag.strip() for tag in tags])
print('Цена:', price)

Название: Party Animals
Категория: All Games > Action Games
Число обзоров: (27,758)
Общая оценка: Mostly Positive
Дата выхода: 20 Sep, 2023
Разработчик: Recreate Games
Метки: ['Casual', 'Funny', 'Multiplayer', 'Cute', 'Party Game', 'Action', 'Fighting', 'PvP', 'Co-op', 'Party', 'Family Friendly', 'Online Co-Op', 'Physics', '3D Fighter', '4 Player Local', 'Local Multiplayer', 'Indie', 'Team-Based', 'Local Co-Op', 'Action-Adventure']
Цена: 433,30 pуб.


Теперь собираем всё это в файлик SteamSpider.py и можно запускать паука. Отдельно в pipelines прописан класс SpiderPipeline.